# Experiment A: Bound Adherence Under Nonstationary Conditions
*Question: does empirical average regret stay within the $\gamma$-regret bounds?*

### Experiment Design
Grid search over the streams (stationary, slow, abrupt, periodic), lambda levels, and two deletion ratio regimes.

Use a static comparator and calibrated learning rate schedule.
Set gamma per-theory at calibration and fix the total horizon (ie. $T=50000$).

### Primary Analysis
For each of the experimental cells, evaluate the final $R_{T}/T$ to see whether the guarantee was met.
Multiple seeds per cell in order to make some kind of causal analysis on the results.

### Success Criteria
95% of the cells meet their guarantees then I would call that a success. Which cells experience more successes than others? Can I make a heatmap that shows the regret results per cell?

## Import Statements and Versioning

This uses pretty standard library imports, but the torch requirement can stress the memory limits of a host.

In [ ]:
# set global seed
import random
import numpy as np
import torch

def set_seed(seed):
    """
    Set the random seed for reproducibility.
    
    Args:
        seed (int): The seed value to set for random number generation.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# Config and Target

In [ ]:
RUN_ID = "experiment_a_test"
REGIME = "stationary"
REPLICATE = 1
T = 1000
SEED = 42
LOSS_NAME = "logistic"
MODEL_NAME = "memorypair"
LBFGS_MEM = 10

# Setup and Imports

In [ ]:
# Import required modules
import sys
import os

# Add code path for imports
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), "..", "code"))

from config import Config
from runner import ExperimentRunner

# Test the d_max Fix

This section demonstrates that the d_max None value error has been fixed.

In [ ]:
# Test parameters that previously caused the error
test_params = {
    'gamma_bar': 1.0,
    'gamma_split': 0.3,
    'accountant': 'zcdp',
    'rho_total': 1.0,
    'delta_total': 1e-05,
    'max_events': 1000,  # Reduced for testing
    'delete_ratio': 0.05,
    'comparator': 'static',
    'd_max': None,  # This was causing the error
    'bootstrap_iters': 50,
    'quantile': 0.95,
    'dataset': 'synthetic',
    'seeds': 2
}

# Create configuration
set_seed(SEED)
cfg = Config.from_cli_args(**test_params)
print(f"Configuration created successfully!")
print(f"d_max value: {cfg.d_max} (should be inf, not None)")

# Create and test runner
runner = ExperimentRunner(cfg)
print(f"ExperimentRunner created successfully!")

# Run one seed to verify the fix
print(f"Running experiment for seed {SEED}...")
result = runner.run_one_seed(seed=SEED)
print(f"✅ Experiment completed successfully! No d_max comparison error.")
print(f"Result type: {type(result)}")

# Summary

The d_max None value comparison error has been fixed. The configuration now properly handles None values by converting them to the default value (float('inf')) for non-Optional fields.